In [ ]:
import sqlite3
db = sqlite3.connect("E:/Movie-Recommender-master/Item-Based Collaborative filtering fix/Accuracy Test/flaskr.sqlite")

In [1]:
page=1
from matplotlib import image
from sklearn.decomposition import TruncatedSVD

from scipy.sparse.linalg import svds
import sqlite3
import pandas as pd
import numpy as np
import requests
def accuracy_test(user):
    db = sqlite3.connect("E:/Movie-Recommender-master/Item-Based Collaborative filtering fix/Accuracy Test/testing.sqlite")
    #Ambil data ratings dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
    ratings = pd.read_sql_query('SELECT userId, movieId, rating FROM ratings',db)
    ratings.columns = ['USERID', 'MOVIEID', 'RATING']
    #Ambil data movies dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
    movies = pd.read_sql_query('SELECT * FROM movies', db)
    movies.columns = ['MOVIEID', 'TITLE', 'GENRES', 'POSTER']

    num_rated = len(db.execute(
        'SELECT m.id, m.title, m.genres, m.poster, r.rating '
        'FROM movies m '
        'INNER JOIN '
        'ratings r '
        'ON r.userID = '+str(user)+' '
        'and m.id = r.movieID '
        'LIMIT 15').fetchall())

    combined_movies_data = pd.merge(ratings, movies, how='inner', on='MOVIEID')#merge data rating dan data movie pada movieId
    #Buat pivot table untuk rating menggunakan userId sebagai index dan movieId sebagai kolom
    rating_crosstab = combined_movies_data.pivot_table(index='USERID', columns='MOVIEID', values='RATING').fillna(0)
    #lakukan transpose dan dimesionality reduction dengan menggunakan fungsi SVD ke tabel pivot yang telah dibuat
    X = rating_crosstab.T
    SVD = TruncatedSVD(n_components=12, random_state=5)
    resultant_matrix = SVD.fit_transform(X)

    #cari korelasi antar film menggunakan pearson correlation menggunakan fungsi corrcoef pada library numpy
    corr_mat = np.corrcoef(resultant_matrix)
    corr_mat = pd.DataFrame(corr_mat, columns = rating_crosstab.columns.tolist(), index = rating_crosstab.columns.tolist())
    #cari data user yang akan diberi rekomendasi berdasarkan rating film tertinggi yang diberikan 
    userId = user
    unrated=X[userId].loc[X[userId]==0]
    rated=X[userId].loc[X[userId]>0]
    #buat pivot table dengan menggunakan movieId yang sudah dirating sebagai kolom dan movieId yang belum dirating sebagai index
    r_ur_corr_matt=corr_mat[rated.index].loc[unrated.index]\
                .apply(lambda x: x.nlargest(10), axis=1).fillna(0)#cari nilai similarity pada 10 item yang sudah 
    simlilarity_dot_weight=r_ur_corr_matt.apply(lambda x: sum((x*rated).fillna(0)), axis=1)
    simlilarity_sum=r_ur_corr_matt.apply(np.sum, axis=1)
    pred=(simlilarity_dot_weight/simlilarity_sum).sort_values(ascending=False)
    reccomendation=pd.concat([movies.set_index('MOVIEID'), pred], axis=1).sort_values(by=0, ascending=False)
    reccomendation.columns = ['title', 'genres', 'poster', 'pred_rating']
    reccomendation=reccomendation.rename_axis('id')
    return reccomendation


In [2]:
user = 249
pred = accuracy_test(user)
pred.loc[[84942, 31696, 84944, 91094, 3034, 84954, 138204, 80862, 3039, 62434, 5093, 7143, 95207, 7153, 134130, 64497, 101362, 44022, 35836, 54272]].to_excel("pred"+str(user)+".xlsx")

In [3]:
pred.loc[[15, 19, 47, 70, 110, 154, 156, 202, 225, 260, 265, 337, 382, 446, 475, 587, 596, 639, 781, 898]]

,title,genres,poster,pred_rating
id,,,,
15,Cutthroat Island (1995),Action|Adventure|Romance,https://image.tmdb.org/t/p/w500/hYdeBZ4BFXivdo...,3.148177
19,Ace Ventura: When Nature Calls (1995),Comedy,https://image.tmdb.org/t/p/w500/wcinCf1ov2D6M3...,2.802634
47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,https://image.tmdb.org/t/p/w500/69Sns8WoET6Cfa...,4.204150
70,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller,https://image.tmdb.org/t/p/w500/sV3kIAmvJ9tPz4...,3.442310
110,Braveheart (1995),Action|Drama|War,https://image.tmdb.org/t/p/w500/or1gBugydmjToA...,3.398872
154,Beauty of the Day (Belle de jour) (1967),Drama,https://image.tmdb.org/t/p/w500/iUAFECovwPA0cV...,4.349358
156,Blue in the Face (1995),Comedy|Drama,https://image.tmdb.org/t/p/w500/yBFNJA95MNiasd...,3.450127
202,Total Eclipse (1995),Drama|Romance,https://image.tmdb.org/t/p/w500/e0j1zDF3ITeTbG...,NaN
225,Disclosure (1994),Drama|Thriller,https://image.tmdb.org/t/p/w500/dnA4QkIZZCF9wR...,3.249058


In [ ]:
t=(simlilarity_dot_weight/simlilarity_sum).sort_values(ascending=False)

In [87]:
movies = pd.read_sql_query('SELECT * FROM movies', db)

In [94]:
rating_crosstab.columns==52831

array([False, False, False, ..., False, False, False])

In [89]:
movies[movies['id']==52831]

,id,title,genres,poster
6475,52831,Maniac Cop (1988),Action|Crime|Horror|Thriller,https://image.tmdb.org/t/p/w500/kFml1HbZlsekXO...


In [4]:
db = sqlite3.connect("E:/Movie-Recommender-master/Item-Based Collaborative filtering fix/Accuracy Test/testing.sqlite")
#Ambil data ratings dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
ratings = pd.read_sql_query('SELECT userId, movieId, rating FROM ratings',db)
ratings.columns = ['USERID', 'MOVIEID', 'RATING']
#Ambil data movies dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
movies = pd.read_sql_query('SELECT * FROM movies', db)
movies.columns = ['MOVIEID', 'TITLE', 'GENRES', 'POSTER']
combined_movies_data = pd.merge(ratings, movies, how='inner', on='MOVIEID')#merge data rating dan data movie pada movieId
unique_movie_rated=combined_movies_data.groupby('USERID').nunique()


In [5]:
unique_movie_rated

,MOVIEID,RATING,TITLE,GENRES,POSTER
USERID,,,,,
1,238,5,238,124,238
2,29,7,29,23,29
3,39,7,39,30,39
4,216,5,216,89,216
5,44,5,44,33,44
...,...,...,...,...,...
606,1115,10,1115,260,1115
607,187,5,187,106,187
608,831,10,831,274,831


In [5]:
more_than_one=combined_movies_data.groupby('MOVIEID')['USERID'].nunique().loc[lambda x: x > 1].index.to_list()
rated=combined_movies_data[combined_movies_data['USERID']==249]["MOVIEID"].to_list()
to_test=list(set(rated) & set(more_than_one))

In [7]:
to_test[543:]

[160718,
 84942,
 31696,
 84944,
 91094,
 3034,
 84954,
 138204,
 80862,
 3039,
 62434,
 5093,
 7143,
 95207,
 7153,
 134130,
 64497,
 101362,
 44022,
 35836,
 54272,
 115713,
 33794,
 1029,
 7173,
 111622,
 3081,
 3082,
 109578,
 1036,
 54286,
 97304,
 97306,
 85020,
 5151,
 1061,
 74789,
 3114,
 74795,
 33836,
 56367,
 5171,
 68659,
 1079,
 1080,
 52281,
 1086,
 85056,
 1089,
 1097,
 3147,
 1101,
 95309,
 7254,
 93270,
 93272,
 142422,
 5218,
 5219,
 5220,
 64614,
 3175,
 109673,
 103539,
 111743,
 31878,
 5254,
 117895,
 85131,
 93326,
 111759,
 5266,
 148626,
 7317,
 101525,
 158872,
 142488,
 27801,
 7323,
 1178,
 7325,
 87192,
 44191,
 79008,
 44195,
 5283,
 111781,
 44199,
 1193,
 1196,
 38061,
 1198,
 1199,
 1200,
 48304,
 1201,
 1203,
 7346,
 93363,
 1206,
 87222,
 68791,
 1209,
 1210,
 68793,
 160954,
 1213,
 83134,
 1214,
 58559,
 87232,
 7361,
 74946,
 1219,
 1220,
 1221,
 3268,
 3271,
 103624,
 3273,
 187593,
 3275,
 64716,
 7373,
 5323,
 95441,
 79057,
 3285,
 1240,
 1343

In [26]:
len(rated)

1260